In [ ]:
import numpy as np

def AttnCoeffs(E, material):
    # Linear attenuation coefficients [cm^-1]
    if material == 'Aluminum':
        muT = 0.08*(1+1/E+0.25/E**2.8)
        muC = 0.2*(1-0.49*np.log(E))
    elif material == 'Lead':
        muT = 0.34*(1+1/E+0.25/E**2.8)
        muC = 0.57*(1-0.49*np.log(E))
    elif material == 'Iron':
        muT = 0.16*(1+1/E+0.25/E**2.8)
        muC = 0.3*(1-0.49*np.log(E))
    elif material == 'Tungsten':
        muT = 0.39*(1+1/E+0.25/E**2.8)
        muC = 0.65*(1-0.49*np.log(E))
    else:
        raise ValueError("Invalid material")

    return muT, muC

def calculate_buildup_factor(material, muL, E0, N):
    # Initialise attenuation coefficients and actual thickness L [cm]
    muT0, muC0 = AttnCoeffs(E0, material)
    L = muL / muT0

    # Create storage space for exit energies
    energy = np.zeros(N)

    # Loop through N photons
    for p in range(N):
        # Initial Energy and attenuation coefficients
        E, muT, muC = E0, muT0, muC0

        # Initial direction coordinates (uvec) and step size (x)
        uvec = np.array([1, 0, 0])
        x = -np.log(np.random.rand()) / muT0

        # Follow the photon’s random walk
        keepgoing = True  # Flag to indicate if photon is still inside slab
        while keepgoing:
            uvec_old = uvec
            if x > L:  # Photon has passed through slab
                energy[p] = E
                keepgoing = False
            elif x < 0:  # Photon has returned to start
                keepgoing = False
            elif np.random.rand() < 1 - muC / muT:  # Photon has been absorbed
                keepgoing = False
            else:  # Photon has been scattered
                # Random angles
                psi = 2 * np.pi * np.random.rand()
                cphi = 1 - 2 * np.random.rand()
                phi = np.arccos(cphi)
                sphi = np.sin(phi)

                # Update unit direction vector
                uvec_rnd = np.array([sphi * np.cos(psi), sphi * np.sin(psi), cphi])
                uvec = uvec_old + uvec_rnd
                uvec = uvec / np.linalg.norm(uvec)

                # cos(theta)
                ctheta = uvec_old @ uvec

                # Update scattered photon energy, attenuation coefficients, and distances
                E = 0.511 / (1 - ctheta + 0.511 / E)
                muT, muC = AttnCoeffs(E, material)
                r = -np.log(np.random.rand()) / muT
                x = x + r * uvec[0]

    # Build-up factor
    BF = np.sum(energy) / (E0 * N * np.exp(-muL))

    return BF

# Parameters
muL = 2.0  # Thickness of the slab in mean free paths
E0 = 1.0  # Energy of incident gamma photons [MeV]
N = 10**6  # Number of gamma photons

# Calculate buildup factors for different materials
materials = ['Aluminum', 'Lead', 'Iron', 'Tungsten']
for material in materials:
    buildup_factor = calculate_buildup_factor(material, muL, E0, N)
    print(f'Buildup Factor for {material}: {buildup_factor}')

# Calculate final intensity for each material
final_intensity = np.exp(-muL) * np.array([calculate_buildup_factor(material, muL, E0, N) for material in materials])
for i, material in enumerate(materials):
    print(f'Final Intensity for {material}: {final_intensity[i]}')

Buildup Factor for Aluminum: 2.10357867425468
Buildup Factor for Lead: 1.6984152469326796
